In [2]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

#Initialize the GUI application
root = tk.Tk()
root.title("Hand Gesture Detection")

video_label = tk.Label(root)
video_label.pack()

#Create textbox for the hand position
gesture_name_var = tk.StringVar()
gesture_name_entry = tk.Entry(root, textvariable=gesture_name_var, font=('Arial', 14), state='readonly')
gesture_name_entry.pack()

#Access webcam
cap = cv2.VideoCapture(0)

def filter_skin(image):
    #Convert to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    lower_bound = np.array([0, 133, 77], dtype="uint8")
    upper_bound = np.array([255, 173, 127], dtype="uint8")
    
    #Create mask to find skin color
    mask = cv2.inRange(ycrcb, lower_bound, upper_bound)
    skin_region = cv2.bitwise_and(image, image, mask=mask)
    
    return skin_region


           
def detect_hand_gesture(image):
    #preprocess image
    skin_region = filter_skin(image)
    gray = cv2.cvtColor(skin_region, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    
    frame_area = image.shape[0] * image.shape[1]
    contours = [cnt for cnt in contours if 0.01 < cv2.contourArea(cnt) / frame_area < 0.1]

    for contour in contours:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) > 3:
            defects = cv2.convexityDefects(contour, hull)
            if defects is not None:
                cnt_defects = 0
                #count defects
                for i in range(defects.shape[0]):  
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])

                    #calculate angle 
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  
                    
                    if angle <= np.radians(90): 
                        cnt_defects += 1
                
                #L shape detection
                l_shape_defects_count = 0
                peace_sign = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                    
                    #Look for angle, L shape vs peace_sign
                    if np.degrees(angle) >= 80 and np.degrees(angle) <= 100:
                        l_shape_defects_count += 1
                    elif np.degrees(angle) <= 55:
                        peace_sign+=1
                
                hull_area = cv2.contourArea(cv2.convexHull(contour))
                solidity = cv2.contourArea(contour) / hull_area
                
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w / float(h)
                
                #Cases for the different hand shapes
                if l_shape_defects_count == 1 and cnt_defects <= 1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2) 
                    return "L Shape Detected", image
                elif peace_sign == 1 and cnt_defects <=1:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 165, 0), 2) 
                    return "Peace sign detected", image
                elif cnt_defects > 3 and solidity < 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2) 
                    return "Five Fingers Detected", image
                elif cnt_defects <= 2 and 0.75 < aspect_ratio < 1.25 and solidity > 0.75:
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  
                    return "Fist Detected", image

    return "No Gesture Detected", image

      
def show_frames():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return
    
    #run each frmae thorugh to find the hand gesture
    gesture, frame_with_box = detect_hand_gesture(frame)

    gesture_name_var.set(gesture)
    
 
    cv2image = cv2.cvtColor(frame_with_box, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)
    
    video_label.after(20, show_frames)

#Start GUI
show_frames()
root.mainloop()

#Release camera 
cap.release()
